In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns

Duplicate key in file WindowsPath('c:/Users/USER/anaconda3/envs/python3.9.15/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 273 ('font.monospace:  DejaVu Sans Mono, Bitstream Vera Sans Mono, Computer Modern Typewriter, Andale Mono, Nimbus Mono L, Courier New, Courier, Fixed, Terminal, monospace')


In [90]:
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
course_df = pd.read_csv('./data/courses.csv')
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

In [4]:
def check_row_number_with_unique_amount(df, df_name, column):
    nrow = df.shape[0]
    nunique = pd.unique(df.loc[:,column]).shape[0]
    msg = f'dataframe {df_name} row number != unique {column} amount'
    assert nrow == nunique, msg
    print(f'dataframe {df_name} row number == unique {column} amount ({nrow} == {nunique})')

def check_unique_items_are_same_set(df1, df1_name, df1_column, df2, df2_name, df2_column):
    df1_set = set(df1.loc[:,df1_column].to_list())
    df2_set = set(df2.loc[:,df2_column].to_list())
    diff = df1_set.symmetric_difference(df2_set)
    msg = f'unique {df1_name}.{df1_column} & unique {df2_name}.{df2_column} are different'
    assert len(diff) == 0, msg
    print(f'unique {df1_name}.{df1_column} & unique {df2_name}.{df2_column} are the same')


# Use LightFM model dataset

## Build json file in format("user_id", "course_id")

In [5]:
users_df.info()
train_df.info()
val_seen_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130566 entries, 0 to 130565
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            130566 non-null  object
 1   gender             85371 non-null   object
 2   occupation_titles  29056 non-null   object
 3   interests          82756 non-null   object
 4   recreation_names   31935 non-null   object
dtypes: object(5)
memory usage: 5.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59737 entries, 0 to 59736
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59737 non-null  object
 1   course_id  59737 non-null  object
dtypes: object(2)
memory usage: 933.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7748 entries, 0 to 7747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    7748 non

### Build Train Data

In [6]:
train_data_df = train_df
train_data_df['course_id'] = train_df['course_id'].apply(lambda x: [i for i in x.split(' ')])
train_data_df.set_index('user_id')['course_id'].apply(pd.Series).reset_index()

data_train_split = pd.melt(train_data_df.set_index('user_id')['course_id'].apply(pd.Series).reset_index(), 
             id_vars=['user_id'],
             value_name='course_id') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['user_id', 'course_id']) \
    .agg({'course_id': 'count'}) \
    .rename(columns={'course_id': 'have_purchased'}) \
    .reset_index() \
    .rename(columns={'course_id': 'course_id'})

data_train_split

,user_id,course_id,have_purchased
0,54dca4456d7d350900e86bae,56f521864e3ef90900b82c12,1
1,54dca4456d7d350900e86bae,601d03e07e6747ef11bba066,1
2,54e421bac5c9c00900cd8d47,6030c9cd99e14cc2401e66b9,1
3,54e961d4c5c9c00900cd8d84,575045e7206cd60900c53085,1
4,54e961d4c5c9c00900cd8d84,6014e3a6910e49742061f87e,1
...,...,...,...
139603,62bd5edfc2d8b50006215c3c,60f83c44b355920006abbd3f,1
139604,62e09de8fc3d3500060d4211,5f7c209762ad22756c7a1c74,1
139605,62e09de8fc3d3500060d4211,5f7c210b1de7982fb413a3e9,1
139606,62e09de8fc3d3500060d4211,5f7c212262ad2203e77a1cc9,1


### Build Valid Data

In [7]:
val_seen_data_df = pd.read_csv('./data/val_seen.csv')
val_seen_data_df['course_id'] = val_seen_df['course_id'].apply(lambda x: [i for i in x.split(' ')])
val_seen_data_df.set_index('user_id')['course_id'].apply(pd.Series).reset_index()

data_val_seen_split = pd.melt(val_seen_data_df.set_index('user_id')['course_id'].apply(pd.Series).reset_index(), 
             id_vars=['user_id'],
             value_name='course_id') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['user_id', 'course_id']) \
    .agg({'course_id': 'count'}) \
    .rename(columns={'course_id': 'have_purchased'}) \
    .reset_index() \
    .rename(columns={'course_id': 'course_id'})

data_val_seen_split

,user_id,course_id,have_purchased
0,54e961d4c5c9c00900cd8d84,5b3b080baa5ff8001e22316e,1
1,54e961d4c5c9c00900cd8d84,5e2a980ee3832b00247239d0,1
2,54e961d4c5c9c00900cd8d84,60dd6d80638ed00aa99b5556,1
3,54ef34e2b839040f00016086,6156a77fdf426a0007cc5fe1,1
4,54ef3574b839040f00016090,5d42554a2fb5020020621025,1
...,...,...,...
17812,614d65d5e7a5df000756b5a4,5a0be0f7a15b3f001e9842ec,1
17813,614d65d5e7a5df000756b5a4,5ff2a7d52a0eb0da5d2c898d,1
17814,615982c5a40fbc0006377d7a,5e46b7f85b02f600245ade9b,1
17815,618b59cd152ad900060f4441,60dd6d80638ed00aa99b5556,1


In [8]:
data_train_val_df = pd.concat([data_train_split, data_val_seen_split], ignore_index=True)
data_train_val_df

,user_id,course_id,have_purchased
0,54dca4456d7d350900e86bae,56f521864e3ef90900b82c12,1
1,54dca4456d7d350900e86bae,601d03e07e6747ef11bba066,1
2,54e421bac5c9c00900cd8d47,6030c9cd99e14cc2401e66b9,1
3,54e961d4c5c9c00900cd8d84,575045e7206cd60900c53085,1
4,54e961d4c5c9c00900cd8d84,6014e3a6910e49742061f87e,1
...,...,...,...
157420,614d65d5e7a5df000756b5a4,5a0be0f7a15b3f001e9842ec,1
157421,614d65d5e7a5df000756b5a4,5ff2a7d52a0eb0da5d2c898d,1
157422,615982c5a40fbc0006377d7a,5e46b7f85b02f600245ade9b,1
157423,618b59cd152ad900060f4441,60dd6d80638ed00aa99b5556,1


In [9]:
(x for x in data_train_val_df['user_id'])

<generator object <genexpr> at 0x000001CF5485D4A0>

## Build dataset and Mapping

In [322]:
import json
from lightfm.data import Dataset

interest_list = ['設計', '投資理財', '行銷', '藝術', '職場技能', '程式', '音樂', '攝影', '語言', '生活品味', '手作', '人文']
gender_list = ['male', 'female']
features_list = interest_list + gender_list

subgroup_num = len(subgroups_df)

dataset = Dataset()
# use train and valid item ids
dataset.fit((x for x in users_df['user_id']),
            (x for x in course_df['course_id']),
            user_features = (x for x in features_list),
            item_features = (str(x) for x in range(1, subgroup_num+1)))


In [323]:
user_id_mapping, user_feature_mapping, item_id_mapping, item_feature_mapping = dataset.mapping()
id_user_mapping = {v : k for k, v in user_id_mapping.items()}
id_item_mapping = {v : k for k, v in item_id_mapping.items()}
item_feature_mapping

{'61888e868f154b000781b45a': 0,
 '54d5a117065a7e0e00725ac0': 1,
 '54d5d9952246e60a009ec571': 2,
 '54d7148a2246e60a009ec588': 3,
 '5513e92b38239d10005778e1': 4,
 '55307d94d530a90a00a3a896': 5,
 '551a6be023774e0a001eb20c': 6,
 '551171a938239d1000577864': 7,
 '54f1268f4ec3c809002e4a29': 8,
 '55599687dfe21b0a00e776d9': 9,
 '559e49185850311000fca504': 10,
 '55a4d6397b4d99100011957d': 11,
 '55ae208a7b4d9910001198f2': 12,
 '55b0ba667b4d991000119a92': 13,
 '557ad62bd736230f00adb2ab': 14,
 '556ad58777a8710900bedffe': 15,
 '55ae36c87b4d99100011990f': 16,
 '55ae34247b4d99100011990d': 17,
 '55ae66017b4d991000119959': 18,
 '5591501e6dec460f00111314': 19,
 '55c636d27b4d99100011a4aa': 20,
 '5593f992cfe8320b00ccd4c4': 21,
 '559a217dcfe8320b00ccd75b': 22,
 '55e3dccbfa223d100058f3cb': 23,
 '5606262ac61d930a00455fad': 24,
 '55e82d8efa223d100058f6f4': 25,
 '55e66f33fa223d100058f5f7': 26,
 '55ea1ea8fa223d100058f7df': 27,
 '5615f2b40eb66b0a00f16f6b': 28,
 '5639fe25423bdd0a00103d55': 29,
 '56082612c61d930a00

In [324]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 130566, num_items 728.


In [325]:
(training_data, weights_train) = dataset.build_interactions(((x, y) for x, y in zip(data_train_split['user_id'], data_train_split['course_id'])))
(valid_data, weights_valid) = dataset.build_interactions(((x, y) for x, y in zip(data_val_seen_split['user_id'], data_val_seen_split['course_id'])))
(valid_data_data, weights_all) = dataset.build_interactions(((x, y) for x, y in zip(data_train_val_df['user_id'], data_train_val_df['course_id'])))
print(repr(training_data))
print(repr(valid_data))
print(repr(valid_data_data))

<130566x728 sparse matrix of type '<class 'numpy.int32'>'
	with 139608 stored elements in COOrdinate format>
<130566x728 sparse matrix of type '<class 'numpy.int32'>'
	with 17817 stored elements in COOrdinate format>
<130566x728 sparse matrix of type '<class 'numpy.int32'>'
	with 157425 stored elements in COOrdinate format>


In [326]:
user_item_matrix = weights_train.toarray()
valid_user_item_matrix = weights_valid.toarray()
all_user_item_matrix = weights_all.toarray()
# for i in range(user_item_matrix.shape[0]):
#     for j in range(user_item_matrix.shape[1]):
#         if user_item_matrix[i][j]:
#             print(f"{id_user_mapping[i]}, {id_item_mapping[j]}")

### Build Feature Matrix

In [327]:
subgroups2idx = {}
for (id, name) in zip(subgroups_df["subgroup_id"], subgroups_df["subgroup_name"]):
    subgroups2idx.update({name: id})

course_df = pd.read_csv('./data/courses.csv')

course2subgroups = {}
for (course_id, sub_groups) in zip(course_df["course_id"], course_df["sub_groups"]):
    if pd.isnull(sub_groups):
        course2subgroups.update({course_id: []})
    else:
        course2subgroups.update({course_id: [ str(subgroups2idx[sub_group]) for sub_group in sub_groups.split(',')]})

list(course2subgroups.items())[0: 5]

[('61888e868f154b000781b45a', ['1', '2']),
 ('54d5a117065a7e0e00725ac0', ['3', '4']),
 ('54d5d9952246e60a009ec571', ['3', '5']),
 ('54d7148a2246e60a009ec588', ['3', '6', '5']),
 ('5513e92b38239d10005778e1', ['7', '8'])]

In [328]:
item_features_matrix = dataset.build_item_features(course2subgroups.items())
print(repr(item_features_matrix))

<728x819 sparse matrix of type '<class 'numpy.float32'>'
	with 1913 stored elements in Compressed Sparse Row format>


In [330]:
user_interests_dict = {}
train_user = train_df["user_id"].to_list()
for i, interests_full_user in enumerate(users_df["interests"]):
    
    user_features_dict = {} 
    if not pd.isnull(interests_full_user): 
        #print(interests_full_user)
        interests_full_user_list = str(interests_full_user).split(",")
        interests_user = []
        for interest_full in interests_full_user_list:
            interests_user.append(str(interest_full).split("_")[0])

        for interest in interests_user:
            user_features_dict[interest] = user_features_dict.setdefault(interest, 0) + 1
        
    if not pd.isnull(users_df["gender"][i]):
        if users_df["gender"][i] == "male":
            user_features_dict.update({"male" : 10})
        else:
            user_features_dict.update({"female" : 10})

    if len(user_features_dict) > 0:
        user_interests_dict.update({users_df["user_id"][i] : user_features_dict})
    
    

list(user_interests_dict.items())[0: 5]


[('54ccaa73a784960a00948687',
  {'職場技能': 2, '藝術': 3, '設計': 3, '投資理財': 1, '行銷': 1, 'female': 10}),
 ('54dca4456d7d350900e86bae',
  {'設計': 4, '程式': 3, '藝術': 3, '音樂': 1, 'male': 10}),
 ('54e421bac5c9c00900cd8d47',
  {'設計': 1, '職場技能': 3, '藝術': 1, '行銷': 1, 'female': 10}),
 ('54e961d4c5c9c00900cd8d84',
  {'投資理財': 3, '攝影': 1, '藝術': 1, '音樂': 2, 'female': 10}),
 ('54e9b744c5c9c00900cd8d8a', {'程式': 2, '投資理財': 3, '設計': 1, 'female': 10})]

In [331]:
user_features_matrix = dataset.build_user_features(user_interests_dict.items())
print(repr(user_features_matrix))

<130566x130580 sparse matrix of type '<class 'numpy.float32'>'
	with 521482 stored elements in Compressed Sparse Row format>


## Train

In [393]:
from lightfm import LightFM

NUM_EPOCHS = 30
ITEM_ALPHA = 0

model = LightFM(loss='warp', learning_rate=0.03, item_alpha=ITEM_ALPHA)
model.fit(training_data, user_features = user_features_matrix, item_features = item_features_matrix, epochs=NUM_EPOCHS, num_threads=1, verbose=True)

Epoch: 100%|██████████| 30/30 [00:20<00:00,  1.45it/s]


## Evaluate

In [394]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

train_precision = precision_at_k(model, training_data, k=10, user_features = user_features_matrix, item_features = item_features_matrix).mean()
test_precision = precision_at_k(model, valid_data, k=10, user_features = user_features_matrix, item_features = item_features_matrix).mean()

train_auc = auc_score(model, training_data, user_features = user_features_matrix, item_features = item_features_matrix).mean()
test_auc = auc_score(model, valid_data, user_features = user_features_matrix, item_features = item_features_matrix).mean()

print('Precision: train %.5f, test %.5f.' % (train_precision, test_precision))
print('AUC: train %.5f, test %.5f.' % (train_auc, test_auc))

Precision: train 0.08620, test 0.03935.
AUC: train 0.92073, test 0.58518.


In [ ]:
Precision: train 0.08463, test 0.04017.
AUC: train 0.91512, test 0.58378.

# Prediction

In [395]:
do_predict = True # False to do validation

In [396]:
if do_predict:
    predict_df = pd.read_csv('./data/test_seen.csv')
    haved_purchased_matrix = all_user_item_matrix
else:
    predict_df = pd.read_csv('./data/val_seen.csv')
    haved_purchased_matrix = user_item_matrix

predict_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    7205 non-null   object
 1   course_id  7205 non-null   object
dtypes: object(2)
memory usage: 112.7+ KB


In [397]:
test_id = predict_df["user_id"].to_list()
test_course = predict_df["course_id"].to_list()
test_seen_info = {}
for id, course in zip(test_id, test_course):
    test_seen_info[id] = course


In [398]:
def sample_recommendation(model, user_ids, user_features_matrix = None, item_features_matrix = None, top = 50):
     
    recommend_list = []
    recommend_list_items_id_list = []
    for user_id in user_ids:
        user_id_index = user_id_mapping[user_id]
        scores = model.predict(user_id_index, np.arange(num_items), user_features = user_features_matrix, item_features = item_features_matrix)   # 698個subgroup
    

        top_items_index = np.argsort(-scores).tolist()
        top_items = []
        for item_index in top_items_index:
            if len(top_items) < top:
                ## 不重複購買
                if haved_purchased_matrix[user_id_index][item_index] != 1: ## Valid: user_item_matrix  Predict: all_user_item_matrix
                    top_items.append(item_index)
                    
        recommend_list.append([id_item_mapping[x] for x in top_items])
        recommend_list_items_id_list.append(top_items)
    return recommend_list, recommend_list_items_id_list

In [399]:
user_id = predict_df["user_id"].to_dict()

users_list =[]
for i in range(len(user_id)):
    users_list.append(user_id[i])

In [401]:
recommend_list, recommend_list_items_id_list = sample_recommendation(model, users_list, user_features_matrix, item_features_matrix, top=50)  # test_seen_daf['user_id'].tolist()

## Save Prediction

In [402]:
import csv
with open("predict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "course_id"])
    for i in range(len(recommend_list)):
        recommend = " ".join(recommend_list[i])
        writer.writerow([user_id[i], recommend])

## MAP 50 evaluate

In [240]:
import ml_metrics
predictions = recommend_list_items_id_list
answers = []
for user in users_list:
    prediction = recommend_list_items_id_list
    answer = []

    user_index = user_id_mapping[user]
    for item_index in range(valid_user_item_matrix.shape[1]):
        if valid_user_item_matrix[user_index][item_index] == 1:
            answer.append(item_index)
    
    answers.append(answer)
        

map50 = ml_metrics.mapk(predictions, answers, k = 50)
map50

0.013327403322772925

# Prediction Unseen
https://stackoverflow.com/questions/46924119/lightfm-handling-user-and-item-cold-start

In [297]:
unseen_predict_df.info()
val_unseen_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11097 entries, 0 to 11096
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    11097 non-null  object
 1   course_id  11097 non-null  object
dtypes: object(2)
memory usage: 173.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11622 entries, 0 to 11621
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    11622 non-null  object
 1   course_id  11622 non-null  object
dtypes: object(2)
memory usage: 181.7+ KB


In [293]:
unseen_predict_df = pd.read_csv('./data/test_unseen.csv')
predict_user = unseen_predict_df["user_id"].to_list()

val_unseen_data_df = pd.read_csv('./data/val_unseen.csv')

unseen_user_haved_purchased_course = {}
for unseen_user_id, course_ids in zip(val_unseen_data_df["user_id"], val_unseen_data_df["course_id"]):
    course_id_list = course_ids.split(' ')
    unseen_user_haved_purchased_course.update({unseen_user_id: course_id_list})
len(unseen_user_haved_purchased_course)

11622

In [288]:
unseen_user_interests_dict = {}

for i, interests_full_user in enumerate(users_df["interests"]):
    if users_df["user_id"][i] not in predict_user: continue
    if pd.isnull(interests_full_user): 
        continue
    
    interests_full_user_list = str(interests_full_user).split(",")
    interests_user = []
    for interest_full in interests_full_user_list:
        interests_user.append(str(interest_full).split("_")[0])

    interests_user_dict = {} 
    for interest in interests_user:
        interests_user_dict[interest] = interests_user_dict.setdefault(interest, 0) + 1

    if len(interests_user_dict) > 0:
        unseen_user_interests_dict.update({users_df["user_id"][i] : interests_user_dict})

list(unseen_user_interests_dict.items())[0: 5]
unseen_user_interests_dict["54eac49bc5c9c00900cd8d95"]


{'行銷': 2, '投資理財': 2}

In [284]:
from scipy import sparse

def get_features(user_feature_dict):
    user_features = np.zeros(len(user_feature_mapping.keys()))
    target_indices = []
    for feature in user_feature_dict.keys():
        try:
            target_indice = user_feature_mapping[feature]
            target_indices.append(target_indice)
            user_features[target_indice] = user_feature_mapping[feature]
        except KeyError:
            #print("new user feature encountered '{}'".format(feature))
            pass
    #print("target indices: {}".format(target_indices))
    user_features = sparse.csr_matrix(user_features)
    return user_features

In [301]:
recommend_list = []
recommend_list_items_id_list = []
for user_id in unseen_predict_df["user_id"]:

    user_feature = np.zeros(len(user_feature_mapping.keys()))
    if user_id in unseen_user_interests_dict.keys():
        user_feature = get_features(unseen_user_interests_dict[user_id])
    else:
        user_feature = sparse.csr_matrix(user_feature)
        
    scores = model.predict(0, np.arange(num_items), user_features = user_feature )   # 698個subgroup


    top_items_index = np.argsort(-scores).tolist()
    top_items = []
    for item_index in top_items_index:
        if len(top_items) < 50:
            ## 不重複購買
            if user_id in unseen_user_haved_purchased_course.keys():
                if id_item_mapping[item_index] not in unseen_user_haved_purchased_course[user_id]: ## Valid: user_item_matrix  Predict: all_user_item_matrix
                    top_items.append(item_index)
            else:
                top_items.append(item_index)
                
    recommend_list.append([id_item_mapping[x] for x in top_items])
    recommend_list_items_id_list.append(top_items)


target indices: [59738, 59742, 59741]
target indices: [59747, 59737, 59741, 59740, 59744, 59738, 59739, 59746, 59748, 59745, 59742, 59743]
target indices: [59744, 59739, 59742, 59738, 59741]
target indices: [59747, 59746, 59740, 59739]
target indices: [59742, 59745]
target indices: [59743, 59745]
target indices: [59746]
target indices: [59739, 59746, 59745, 59741, 59744, 59737, 59740, 59747]
target indices: [59739, 59745, 59742, 59741]
target indices: [59743, 59745, 59747, 59744, 59738, 59746, 59741]
target indices: [59745, 59744, 59747, 59748, 59738, 59746]
target indices: [59746, 59745, 59738]
target indices: [59746, 59738, 59744, 59745, 59741]
target indices: [59744, 59742, 59737, 59743, 59741, 59738, 59746, 59740, 59748, 59739]
target indices: [59747]
target indices: [59743, 59745, 59740]
target indices: [59742]
target indices: [59745, 59738]
target indices: [59747, 59741, 59746, 59744, 59739]
target indices: [59740, 59737]
target indices: [59747, 59742, 59743, 59738, 59746, 59740]

In [303]:
import csv

predict_user = unseen_predict_df["user_id"].to_list()
with open("unseen_predict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "course_id"])
    for i in range(len(recommend_list)):
        recommend = " ".join(recommend_list[i])
        writer.writerow([predict_user[i], recommend])